#### Class: Preprocessor

Input:  

    1. st_log_path: Strace log
    
    2. filtered_calls: List of system calls to be tracked

    3. an instance of the LineReader class

In [79]:
%%writefile ../strace_miner/core/preprocessor.py

import os
import csv
import pandas as pd
from datetime import datetime

class Preprocessor():
    def __init__(self,strace_log_path,filtered_calls,line_reader,name='Filtered'):
         self.st_log = strace_log_path   
         self.csv_log = self.st_log.split('.st')[0]+'.csv'      
         self.filtered_calls = filtered_calls
         self.line_reader = line_reader
         self.name = name
                  
         self.cols = ['pid','call','time','duration']
         self.extra = []
         self.io_time = 0.0
         self.total_time = 0.0
         self.dt_format = "%H:%M:%S.%f"
         self.stat_cols = ['call','duration','percent']


    def prepare_csv_log(self):
        print("CSV log path: {}".format(self.csv_log))

        with open(self.st_log,'r') as in_f, open(self.csv_log,'w') as csv_f:
            csv_writer = csv.writer(csv_f)
            csv_writer.writerow(self.cols+self.extra)
            for line in in_f:
                ret = self.line_reader.process_line(line)
                if ret:
                    csv_writer.writerow(ret)

    def get_df_from_csv(self,csv_file=None):
        if csv_file:
            self.csv_log = csv_file
        if not os.path.exists(self.csv_log):
            return None
        return pd.read_csv(self.csv_log,sep=',')
    
    def _get_time_summary(self,df):
        stats = []
        ##get total time
        start = datetime.strptime(df['time'].iloc[0],self.dt_format)
        end = datetime.strptime(df['time'].iloc[-1],self.dt_format)
        self.total_time = (end-start).total_seconds()
        #print(self.total_time)
        stats.append(['Total',self.total_time,100.0])

        ## get the time spent on all sys calls
        #self.sys_time = df['duration'].sum()
        #stats.append(['Sys time',self.sys_time,(self.sys_time/self.total_time)*100.0,100.0])

        return stats
    
    def process(self,df):
        df['duration'] = df['duration'].apply(lambda x: max(0.0,x))
        summary = self._get_time_summary(df)

        if self.filtered_calls: 
            ## Filter the io_calls and find the time spent on io
            df = df[df['call'].isin(self.filtered_calls)]
            io_time = df['duration'].sum()
            summary.append(['{} time'.format(self.name),io_time,(io_time/self.total_time)*100.0])

            remaining_time = self.total_time - io_time
            summary.append(['Remaining time', remaining_time,
                        (remaining_time/self.total_time)*100.0])

            ## find the total duration of each io call
            grouped = df.groupby('call')
            sum_by_call = dict(grouped['duration'].sum())      
            for call,dur in sum_by_call.items():
                p_tot = (dur/self.total_time)*100.0      
                summary.append([call,dur,p_tot])

        return pd.DataFrame(summary,columns=self.stat_cols),df
    



Overwriting ../strace_miner/core/preprocessor.py


### Test class: 

Sample log

In [52]:
!strace -tt -T -r -e futex python src/mmarrays.py 100000

16:10:05.548847 (+     0.000000) futex(0x7f7d8013b99c, FUTEX_WAKE_PRIVATE, 2147483647) = 0 <0.000037>
16:10:05.616257 (+     0.067390) futex(0x7f7d8028e0c8, FUTEX_WAKE_PRIVATE, 2147483647) = 0 <0.000051>
16:10:05.800275 (+     0.184008) futex(0x7f7cfd088404, FUTEX_WAKE_PRIVATE, 2147483647) = 0 <0.000070>
16:10:05.800456 (+     0.000171) futex(0x7f7cfd087f8c, FUTEX_WAKE_PRIVATE, 2147483647) = 0 <0.000032>
16:10:05.800554 (+     0.000090) futex(0x7f7cfd0883f8, FUTEX_WAKE_PRIVATE, 2147483647) = 0 <0.000031>
16:10:05.800650 (+     0.000096) futex(0x7f7cfd0883f0, FUTEX_WAKE_PRIVATE, 2147483647) = 0 <0.000029>
16:10:05.800735 (+     0.000083) futex(0x7f7cfd088510, FUTEX_WAKE_PRIVATE, 2147483647) = 0 <0.000041>
16:10:05.800957 (+     0.000224) futex(0x7f7cfd0883e0, FUTEX_WAKE_PRIVATE, 2147483647) = 0 <0.000029>
16:10:05.801053 (+     0.000094) futex(0x7f7cfd0883d8, FUTEX_WAKE_PRIVATE, 2147483647) = 0 <0.000029>
16:10:05.801553 (+     0.000505) futex(0x7f7cfd087ec4, FUTEX_WAKE_PRIVATE, 2147483

In [70]:
%%bash
#strace -f -tt -T -r -o logs/mmarrays.st python src/mmarrays.py 100000
strace -f -tt -T -r -o logs/multi_threaded.st python src/multi_threaded.py

In [71]:
#!cat logs/mmarrays.st
!cat logs/multi_threaded.st

71243 16:19:12.768289 (+     0.000000) execve("/opt/conda/envs/hdf5-tutorial/bin/python", ["python", "src/multi_threaded.py"], 0x7ffe878fe708 /* 59 vars */) = 0 <0.000261>
71243 16:19:12.768889 (+     0.000575) brk(NULL) = 0x560669de8000 <0.000146>
71243 16:19:12.769364 (+     0.000467) readlink("/proc/self/exe", "/opt/conda/envs/hdf5-tutorial/bi"..., 4096) = 44 <0.000204>
71243 16:19:12.769791 (+     0.000424) access("/etc/ld.so.preload", R_OK) = -1 ENOENT (No such file or directory) <0.000047>
71243 16:19:12.770007 (+     0.000214) openat(AT_FDCWD, "/opt/conda/envs/hdf5-tutorial/bin/../lib/tls/haswell/x86_64/libpthread.so.0", O_RDONLY|O_CLOEXEC) = -1 ENOENT (No such file or directory) <0.000143>
71243 16:19:12.770300 (+     0.000293) stat("/opt/conda/envs/hdf5-tutorial/bin/../lib/tls/haswell/x86_64", 0x7ffcc638c100) = -1 ENOENT (No such file or directory) <0.000081>
71243 16:19:12.770483 (+     0.000185) openat(AT_FDCWD, "/opt/conda/envs/hdf5-tutorial/bin/../lib/tls/haswell/libpthrea

In [72]:
from strace_miner.core.strace_line_reader import StraceLineReader


In [73]:
LOG_FILE = "logs/multi_threaded.st"
#LOG_FILE = "logs/mmarrays.st"

lr = StraceLineReader()
io_calls = ['read','write','mmap','openat']
st = Preprocessor(LOG_FILE,io_calls,lr,'IO')


Member: Prepare_csv_log

In [74]:
st.prepare_csv_log()

CSV log path: logs/multi_threaded.csv


Member: Get df from csv

In [75]:
df = st.get_df_from_csv()
df

,pid,call,time,duration
0,71243,execve,16:19:12.768289,0.000261
1,71243,brk,16:19:12.768889,0.000146
2,71243,readlink,16:19:12.769364,0.000204
3,71243,access,16:19:12.769791,0.000047
4,71243,openat,16:19:12.770007,0.000143
...,...,...,...,...
7198,71243,munmap,16:19:13.350959,0.000044
7199,71243,munmap,16:19:13.351042,0.000031
7200,71243,munmap,16:19:13.351109,0.000030
7201,71243,exit_group,16:19:13.351293,-2.000000


In [76]:
df['pid'].unique()

array([71243, 71245, 71246, 71247, 71248, 71249, 71250, 71251, 71252,
       71253, 71254, 71255, 71256, 71257, 71258, 71259, 71260, 71261,
       71262, 71263])

Member fn: Process

Output: summary and reduced dataframe

In [77]:
summary,df = st.process(df)
summary

,call,duration,percent
0,Total,0.584828,100.000000
1,IO time,0.116317,19.889096
2,Remaining time,0.468511,80.110904
3,mmap,0.013618,2.328548
4,openat,0.016129,2.757905
5,read,0.014186,2.425670
6,write,0.072384,12.376972


In [78]:
df

,pid,call,time,duration
4,71243,openat,16:19:12.770007,0.000143
6,71243,openat,16:19:12.770483,0.000048
8,71243,openat,16:19:12.770809,0.000155
10,71243,openat,16:19:12.771429,0.000052
12,71243,openat,16:19:12.771708,0.000050
...,...,...,...,...
6992,71261,write,16:19:13.301537,0.014939
6995,71263,write,16:19:13.306843,0.017559
6996,71262,write,16:19:13.303475,0.020984
7008,71261,write,16:19:13.326046,0.000192
